# Tracing with structured logging

This walkthrough guides you through creating a simple agent, and use structured logging to produce traces for easier debugging.


In [8]:
import os
from openai import AzureOpenAI
from actionweaver.llms import patch
from actionweaver.llms.azure.tokens import TokenUsageTracker



client = patch(AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview"
))

In [9]:
import logging
from pythonjsonlogger import jsonlogger

# Initialize logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


# Create a FileHandler for logging to the file 'tracing.log'
file_handler = logging.FileHandler('tracing.log')
logger.addHandler(file_handler)

# Define JSON format
log_format = jsonlogger.JsonFormatter(
    '%(asctime)s.%(msecs)04d %(levelname)s %(module)s %(funcName)s %(message)s %(lineno)d',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Set the JSON formatter for handlers
file_handler.setFormatter(log_format)


In [10]:
from actionweaver import action

@action(name="GetCurrentTime", logger=logger)
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False, logger=logger)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what time is it and what's the weather in San Francisco and Tokyo ?"}
  ]


response = client.chat.completions.create(
  model="gpt-35-turbo-0613-16k",
  messages=messages,
    actions = [get_current_time, get_current_weather],
    stream=False, 
    token_usage_tracker = TokenUsageTracker(5000),
    logger=logger,
    logging_name="conversation_start",
)

response

Getting current time...
Getting current weather
Getting current weather


ChatCompletion(id='chatcmpl-8bIPaEOsveApjS9dnOKVqRCLHNC3F', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current time is 20:26:42.135028. \n\nIn San Francisco, the weather is currently 72°F.\n\nIn Tokyo, the weather is currently 10°C.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1703899602, model='gpt-35-turbo-16k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=39, prompt_tokens=240, total_tokens=279), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence'

**Visualize traces**

In [ ]:
%pip install pyvis==0.3.1

In [12]:
import json
from pyvis.network import Network
import datetime


# Replace this with the actual path to your log file
log_file_path = 'tracing.log'

# Initialize a network graph
net = Network(height="750px", width="100%", directed=True)

# Read the log file and parse it
with open(log_file_path, 'r') as file:
    logs = [json.loads(line.strip()) for line in file]
    logs.sort(key=lambda x: x['timestamp'])
    
    for log_entry in logs:
        run_id = log_entry.get("run_id")
        # Add the current log entry as a node
        net.add_node(run_id, label=log_entry['name'] + " at " + str(datetime.datetime.fromtimestamp(log_entry['timestamp'])))


    for log_entry in logs:
        run_id = log_entry.get("run_id")
        parent_run_id = log_entry.get("parent_run_id")
        if parent_run_id:
            net.add_edge(parent_run_id, run_id)

# Set some options for better visualization (optional)
net.set_options("""
var options = {
  "nodes": {
    "shape": "dot",
    "scaling": {
      "min": 10,
      "max": 30
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -100,
      "centralGravity": 0.01,
      "springLength": 200,
      "springConstant": 0.08
    },
    "maxVelocity": 50,
    "minVelocity": 0.1,
    "solver": "forceAtlas2Based"
  }
}
""")

# Save and show the network
net.show('network.html')


In [ ]:
# explain inital_conversation mean